In [190]:
# Useful starting lines
import numpy as np
from helpers import *
from methods import *
from process_data import *
from crossvalidation import *
from select_parameter import *

%load_ext autoreload
%autoreload 2

seed=20

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [191]:
a=np.array([[1,2,3],[4,5,6],[6,7,8]])
a.shape
build_poly(a,3)

array([[  1,   2,   3,   1,   2,   3,   4,   6,   9,   1,   2,   3,   4,
          6,   9,   8,  12,  18,  27,   1,   8,  27],
       [  4,   5,   6,  16,  20,  24,  25,  30,  36,  64,  80,  96, 100,
        120, 144, 125, 150, 180, 216,  64, 125, 216],
       [  6,   7,   8,  36,  42,  48,  49,  56,  64, 216, 252, 288, 294,
        336, 384, 343, 392, 448, 512, 216, 343, 512]])

# Load the dataset

In [192]:
from zipfile import ZipFile 
  
# # specifying the zip file name 
file_name = 'Data/test.csv.zip'
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    zip.extractall('Data/') 

In [193]:
y, tX, ids = load_csv_data('Data/train.csv')
_, tX_test, ids_test = load_csv_data('Data/test.csv')

In [186]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 23)
pca.fit(tX)
X_train_pca = pca.transform(tX)
print(pca.explained_variance_)
X_train_pca

[1.98361790e+06 4.70787394e+05 1.52814225e+05 3.51276806e+04
 1.81844503e+04 2.23538472e+03 1.83311263e+03 1.26660194e+03
 9.44046982e+02 5.19385510e+02 3.84395271e+02 3.74827197e+02
 1.78779816e+02 1.38811517e+02 4.57897240e+01 3.98920386e+00
 3.49795311e+00 2.74456931e+00 2.45149215e+00 1.65338307e+00
 1.40891360e+00 9.95640451e-01 7.46670055e-01]


array([[ 1.99836109e+03, -2.10173604e+02,  8.14167196e+01, ...,
         2.00564313e+00,  2.88269181e+00, -1.49131148e+00],
       [-4.13388223e+02,  1.00498122e+03,  1.06835313e+02, ...,
         1.54134907e-01,  4.74176942e-02, -1.47560528e+00],
       [-4.79440815e+02,  8.57272034e+02, -1.03988966e+03, ...,
        -6.07796561e-01,  1.22248798e+00, -2.87796566e+00],
       ...,
       [-4.16847076e+02,  9.95937375e+02,  5.32802660e+01, ...,
        -1.83859802e-01,  8.12707811e-01, -4.23211088e-01],
       [-1.21173594e+03, -5.54246847e+02,  2.96851244e+02, ...,
        -1.30280561e-01,  2.23422337e-02, -1.82388641e+00],
       [-1.28148740e+03, -6.94469079e+02, -7.86627104e+02, ...,
         2.13837124e-01, -5.08304783e-01, -5.43329348e-02]])

In [ ]:
c1=np.count_nonzero(tX[:,22]==0)
c2=np.count_nonzero(tX[:,22]==1)
c3=np.count_nonzero(tX[:,22]==2)
c4=np.count_nonzero(tX[:,22]==3)

print(c1,c2,c3,c4)

99913 77544 50379 22164


# Methods

## 1. Least Squares with Gradient Descent

#### Cross Validation

In [ ]:
# Degree polynomial expansion
degrees = [10,10,10,10]

# Model parameters
max_iters = 5000
gamma = 0.005


# Split data in k-fold
k_fold = 2
k_indices = build_k_indices(y, k_fold, seed)


accs_train = []
accs_test = []

for k in range(k_fold):
    acc_train, acc_test = cross_validation_least_squares_GD(y, tX, k_indices, k, max_iters, gamma, degrees)
    accs_train.append(acc_train)
    accs_test.append(acc_test)
    
for i in range(len(accs_train)):
    print("%d - Training accuracy: %f / Test accuracy : %f" % (i, accs_train[i], accs_test[i]))

print("\nAverage test accuracy: %f" % np.mean(accs_test))
print("Variance test accuracy: %f" % np.var(accs_test))
print("Min test accuracy: %f" % np.min(accs_test))
print("Max test accuracy: %f" % np.max(accs_test))

0 - Training accuracy: 0.780448 / Test accuracy : 0.781016
1 - Training accuracy: 0.779968 / Test accuracy : 0.778256

Average test accuracy: 0.779636
Variance test accuracy: 0.000002
Min test accuracy: 0.778256
Max test accuracy: 0.781016


## 2. Least Squares with Stochastic Gradient Descent

#### Cross Validation

In [ ]:
#TO DO

initial_w=np.zeros(tX.shape[1])
batch_size=1
max_iters=1000
gamma=0.005

loss, weights = least_squares_SGD(y, tX, initial_w, batch_size, max_iters, gamma)

## 3. Least Squares with Normal Equations 

#### Cross Validation

In [194]:
# Degree polynomial expansion
#degrees = [8,11,9]
#degrees = [7,10,9,9]
degrees = [5,7,6,6]

alpha = 0

# Split data in k-fold
k_fold = 3
k_indices = build_k_indices(y, k_fold, seed)


accs_train = []
accs_test = []

for k in range(k_fold):
    print(k)
    acc_train, acc_test = cross_validation_least_squares(y, tX, k_indices, k, degrees, alpha)
    accs_train.append(acc_train)
    accs_test.append(acc_test)
    
for i in range(len(accs_train)):
    print("%d - Training accuracy: %f / Test accuracy : %f" % (i, accs_train[i], accs_test[i]))

print("\nAverage test accuracy: %f" % np.mean(accs_test))
print("Variance test accuracy: %f" % np.var(accs_test))
print("Min test accuracy: %f" % np.min(accs_test))
print("Max test accuracy: %f" % np.max(accs_test))

0
1


C:\Users\Riccardo\Desktop\1° Semester\Machine Learning\Projects\higgs_boson_classification\process_data.py:116: RuntimeWarning: divide by zero encountered in log
  x_test_t2 = np.log(x_test[:, inv_log_cols])


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

## 4. Ridge regression with Normal Equations

#### Lambda

In [ ]:
# Model parameters for least squares
#tuning parameters for each category
degrees_candidates = [9,10,11,12,13,14]
alpha = 0
k_fold = 3
par_degree, par_lambda, accu = select_parameters_least_squares(y,tX,degrees_candidates,alpha,k_fold,seed)
par_degree, accu

In [ ]:
# Model parameters for ridge regression
#tuning parameters for each category
degrees_candidates = [9,10,11,12,13,14]
lambdas_candidates = np.logspace(-10, 0, 20)
alpha = 0
k_fold = 3
par_degree, par_lambda, accu = select_parameters_ridge_regression(y,tX,degrees_candidates,lambdas_candidates,alpha,k_fold,seed)
par_degree, par_lambda, accu

#### Cross Validation

In [ ]:
# Process data parameters
#degrees = [8,11,10,10]
degrees = [7,10,9,9]
alpha = 1

# Model parameters
lambdas = [0,0,0,0]

# Split data in k-fold
k_fold = 3
k_indices = build_k_indices(y, k_fold, seed)


accs_train = []
accs_test = []

for k in range(k_fold):
    acc_train, acc_test = cross_validation_ridge_regression(y, tX, k_indices, k, lambdas, degrees, alpha)
    accs_train.append(acc_train)
    accs_test.append(acc_test)
    
for i in range(len(accs_train)):
    print("%d - Training accuracy: %f / Test accuracy : %f" % (i, accs_train[i], accs_test[i]))

print("\nAverage test accuracy: %f" % np.mean(accs_test))
print("Variance test accuracy: %f" % np.var(accs_test))
print("Min test accuracy: %f" % np.min(accs_test))
print("Max test accuracy: %f" % np.max(accs_test))

0 - Training accuracy: 0.825165 / Test accuracy : 0.818787
1 - Training accuracy: 0.835869 / Test accuracy : 0.831627
2 - Training accuracy: 0.838593 / Test accuracy : 0.831351

Average test accuracy: 0.827255
Variance test accuracy: 0.000036
Min test accuracy: 0.818787
Max test accuracy: 0.831627


## 5. Logistic Regression with Stochastic Gradient Descent


#### Cross Validation

In [ ]:
# Degree polynomial expansion
degrees = [7,10,9,9]

# Model parameters
max_iters = 5000
gamma = 0.005
batch_size = 1

# Split data in k-fold
k_fold = 2
k_indices = build_k_indices(y, k_fold, seed)


accs_train = []
accs_test = []

for k in range(k_fold):
    acc_train, acc_test = cross_validation(y, tX, logistic_regression, k_indices, k, batch_size=batch_size, 
                                           max_iters=max_iters, gamma=gamma, degrees=degrees)
    accs_train.append(acc_train)
    accs_test.append(acc_test)
    
for i in range(len(accs_train)):
    print("%d - Training accuracy: %f / Test accuracy : %f" % (i, accs_train[i], accs_test[i]))

print("\nAverage test accuracy: %f" % np.mean(accs_test))
print("Variance test accuracy: %f" % np.var(accs_test))
print("Min test accuracy: %f" % np.min(accs_test))
print("Max test accuracy: %f" % np.max(accs_test))

7.997182550126146 0 /5000
0.7524469806959074 1000 /5000
0.4363734465688057 2000 /5000
0.400023839876429 3000 /5000
0.40076385746875537 4000 /5000
10.371044956652037 0 /5000
1.5550707519015254 1000 /5000
0.661663323193932 2000 /5000
0.5936779549816508 3000 /5000
0.5260682969312651 4000 /5000
8.442467521375363 0 /5000
1.7929335626679517 1000 /5000
0.7223707415043099 2000 /5000
0.5914451124506169 3000 /5000
0.5454690010169485 4000 /5000
12.029170671306018 0 /5000
2.0525746152729285 1000 /5000
0.8818547205280977 2000 /5000
0.6671236655053144 3000 /5000
0.5709404738188759 4000 /5000
6.689309726685983 0 /5000
0.7984062586900572 1000 /5000
0.4669079564079578 2000 /5000
0.4200246548981903 3000 /5000
0.4292098025507569 4000 /5000
9.748515728107092 0 /5000
1.2804452477863466 1000 /5000
0.6366004683073873 2000 /5000
0.5702327086476892 3000 /5000
0.5406348712757687 4000 /5000
10.207457083659229 0 /5000
1.8709968020623324 1000 /5000
0.8871537214879525 2000 /5000
0.6549933484898737 3000 /5000
0.6273

## 6. Regularized Logistic Regression with Stochastic Gradient Descent

#### Optimal Lambda

In [ ]:
# TO DO

#### Cross Validation

In [ ]:
# TO DO

lambda_ = 0.001
initial_w = np.random.random(tX.shape[1])
batch_size = 1
max_iters = 1000
gamma = 0.1

loss, weights = reg_logistic_regression(y, tX, lambda_, initial_w, batch_size,  max_iters, gamma)

# Prediction (file.run)
by now the best accuracy predicted is through RIDGE REGRESSION

In [176]:
# Split data in subsets corresponding to a jet value
msks_jet_train = get_jet_masks(tX)
msks_jet_test = get_jet_masks(tX_test)

# Degree polynomial expansion
degrees = [5,7,6,6]
alpha = 0
# Ridge regression parameters for each subset
lambdas = [0,0,0,0]

# Vector to store the final prediction
y_pred = np.zeros(tX_test.shape[0])

for idx in range(len(msks_jet_train)):
    x_train = tX[msks_jet_train[idx]]
    x_test = tX_test[msks_jet_test[idx]]
    y_train = y[msks_jet_train[idx]]

    # Pre-processing of data
    x_train, x_test = process_data(x_train, x_test, alpha)
    x_train, x_test = phi(x_train, x_test, degrees[idx])

    loss, weights = ridge_regression(y_train, x_train, lambdas[idx])

    y_test_pred = predict_labels(weights, x_test)

    y_pred[msks_jet_test[idx]] = y_test_pred

In [177]:
higgs = np.count_nonzero(y_pred==1)
print(f'From {y_pred.shape[0]} test examples, {higgs} are 1, i.e. the {higgs/y_pred.shape[0]} %')

From 568238 test examples, 181823 are 1, i.e. the 0.31997684068999255 %


#### Generate predictions and save ouput in csv format for submission

In [178]:
OUTPUT_PATH = 'data/leastSquares.csv' 
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

# OTHERS (old)

### Umbalanced Dataset

In [ ]:
higgs = np.count_nonzero(y==1)
print(f'From {y.shape[0]} training examples, {higgs} are 1, i.e. the {higgs/y.shape[0]} %')

# Random Over Sampling
#tX, y = Random_Over_Sampling(tX, y)

#higgs = np.count_nonzero(y==1)
#print(f'Applying Random Over Sampling: \nFrom {y.shape[0]} training examples, {higgs} are 1, i.e. the {higgs/y.shape[0]} %')

# Preprocessing

In [ ]:
tX, tX_test = process_data(tX, tX_test, add_constant_col=True)

# Cross Validation
IDEA: insert CV in each of the methods above

In [ ]:
def cross_validation(y, x, k_indices, k, regression_method, **args):
    """
    Completes k-fold cross-validation using the regression method
    passed as argument.
    """
    # get k'th subgroup in test, others in train
    msk_test = k_indices[k]
    msk_train = np.delete(k_indices, (k), axis=0).ravel()

    x_train = x[msk_train, :]
    x_test = x[msk_test, :]
    y_train = y[msk_train]
    y_test = y[msk_test]

    # data pre-processing
    #x_train, x_test = process_data(x_train, x_test, True)

    # compute weights using given method
    loss, weights = regression_method(y=y_train, tx=x_train, **args)
    
    # predict output for train and test data
    y_train_pred = predict_labels(weights, x_train)
    y_test_pred = predict_labels(weights, x_test)
    
    
    # compute accuracy for train and test data
    acc_train = compute_accuracy(y_train_pred, y_train)
    acc_test = compute_accuracy(y_test_pred, y_test)

    return acc_train, acc_test

In [ ]:
regression_method = ridge_regression

# Model parameters
lambda_ = 0.0005

# Split data in k-fold
k_fold = 2
k_indices = build_k_indices(y, k_fold, seed)


accs_train = []
accs_test = []

for k in range(k_fold):
    acc_train, acc_test = cross_validation(y, tX, k_indices, k, regression_method, lambda_=lambda_)
    accs_train.append(acc_train)
    accs_test.append(acc_test)
    
for i in range(len(accs_train)):
    print("%d - Training accuracy: %f / Test accuracy : %f" % (i, accs_train[i], accs_test[i]))

print("\nAverage test accuracy: %f" % np.mean(accs_test))
print("Variance test accuracy: %f" % np.var(accs_test))
print("Min test accuracy: %f" % np.min(accs_test))
print("Max test accuracy: %f" % np.max(accs_test))

0 - Training accuracy: 0.775480 / Test accuracy : 0.776096
1 - Training accuracy: 0.775888 / Test accuracy : 0.774656

Average test accuracy: 0.775376
Variance test accuracy: 0.000001
Min test accuracy: 0.774656
Max test accuracy: 0.776096


In [ ]:
# TO CHECK

# To evaluate the best lambda that minimizes the test error
loss, weights, best_lambda = cross_validation_ridge_regression(y,tX)

In [ ]:
# Only for non logistic methods
y_pred = predict_labels(weights, tX_test)

In [ ]:
# Only for Logistic methods
y_pred = sigmoid(tX_test@weights)
y_pred[y_pred <0.5] = -1
y_pred[y_pred > 0.5] = 1